# 🎄 Santa 2025 — Optimization Trick

We take each box and try to remove one by one trees which touches bbox

In [ ]:
import pandas as pd
from decimal import Decimal, getcontext
from shapely import affinity, touches
from shapely.geometry import Polygon
from shapely.ops import unary_union
from shapely.strtree import STRtree

getcontext().prec = 25
scale_factor = Decimal('1e18')

In [ ]:
class ChristmasTree:
    """Represents a single, rotatable Christmas tree of a fixed size."""

    def __init__(self, center_x='0', center_y='0', angle='0'):
        """Initializes the Christmas tree with a specific position and rotation."""
        self.center_x = Decimal(center_x)
        self.center_y = Decimal(center_y)
        self.angle = Decimal(angle)

        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon(
            [
                # Start at Tip
                (Decimal('0.0') * scale_factor, tip_y * scale_factor),
                # Right side - Top Tier
                (top_w / Decimal('2') * scale_factor, tier_1_y * scale_factor),
                (top_w / Decimal('4') * scale_factor, tier_1_y * scale_factor),
                # Right side - Middle Tier
                (mid_w / Decimal('2') * scale_factor, tier_2_y * scale_factor),
                (mid_w / Decimal('4') * scale_factor, tier_2_y * scale_factor),
                # Right side - Bottom Tier
                (base_w / Decimal('2') * scale_factor, base_y * scale_factor),
                # Right Trunk
                (trunk_w / Decimal('2') * scale_factor, base_y * scale_factor),
                (trunk_w / Decimal('2') * scale_factor, trunk_bottom_y * scale_factor),
                # Left Trunk
                (-(trunk_w / Decimal('2')) * scale_factor, trunk_bottom_y * scale_factor),
                (-(trunk_w / Decimal('2')) * scale_factor, base_y * scale_factor),
                # Left side - Bottom Tier
                (-(base_w / Decimal('2')) * scale_factor, base_y * scale_factor),
                # Left side - Middle Tier
                (-(mid_w / Decimal('4')) * scale_factor, tier_2_y * scale_factor),
                (-(mid_w / Decimal('2')) * scale_factor, tier_2_y * scale_factor),
                # Left side - Top Tier
                (-(top_w / Decimal('4')) * scale_factor, tier_1_y * scale_factor),
                (-(top_w / Decimal('2')) * scale_factor, tier_1_y * scale_factor),
            ]
        )
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated,
                                          xoff=float(self.center_x * scale_factor),
                                          yoff=float(self.center_y * scale_factor))
        

    def clone(self) -> "ChristmasTree":
        return ChristmasTree(
            center_x=str(self.center_x),
            center_y=str(self.center_y),
            angle=str(self.angle),
        )    


def get_tree_list_side_lenght(tree_list: list[ChristmasTree]) -> Decimal:
    all_polygons = [t.polygon for t in tree_list]
    bounds = unary_union(all_polygons).bounds
    return Decimal(max(bounds[2] - bounds[0], bounds[3] - bounds[1])) / scale_factor

def get_total_score(dict_of_side_length: dict[str, Decimal]):
    score = 0
    for k, v in dict_of_side_length.items():
        score += v ** 2 / Decimal(k)
    return score

def parse_csv(csv_path) -> dict[str, list[ChristmasTree]]:
    print(f'parse_csv: {csv_path=}')

    result = pd.read_csv(csv_path)
    result['x'] = result['x'].str.strip('s')
    result['y'] = result['y'].str.strip('s')
    result['deg'] = result['deg'].str.strip('s')
    result[['group_id', 'item_id']] = result['id'].str.split('_', n=2, expand=True)

    dict_of_tree_list = {}
    dict_of_side_length = {}
    for group_id, group_data in result.groupby('group_id'):
        tree_list = [ChristmasTree(center_x=row['x'], center_y=row['y'], angle=row['deg']) for _, row in group_data.iterrows()]
        dict_of_tree_list[group_id] = tree_list
        dict_of_side_length[group_id] = get_tree_list_side_lenght(tree_list)

    return dict_of_tree_list, dict_of_side_length

In [ ]:
from shapely.geometry import box

def get_bbox_touching_tree_indices(tree_list: list[ChristmasTree]) -> list[int]:
    """
    Given a list of trees, this function:

      1. Computes the minimal axis-aligned bounding box around all trees.
      2. Returns the list of indices of trees whose boundaries touch
         the boundary of that bounding box.

    Returns:
        touching_indices: list[int]  -- indices in tree_list
    """

    if not tree_list:
        return []

    # Collect polygons
    polys = [t.polygon for t in tree_list]

    # Compute global bounding box from all polygon bounds
    minx = min(p.bounds[0] for p in polys)
    miny = min(p.bounds[1] for p in polys)
    maxx = max(p.bounds[2] for p in polys)
    maxy = max(p.bounds[3] for p in polys)

    bbox = box(minx, miny, maxx, maxy)

    # Check boundary intersection: only trees touching the box border
    touching_indices = [
        i
        for i, poly in enumerate(polys)
        if poly.boundary.intersects(bbox.boundary)
    ]

    return touching_indices


In [ ]:
# Load current best solution
current_solution_path = '/kaggle/input/santa25-ensemble-sa-fractional-translation/submission.csv'
dict_of_tree_list, dict_of_side_length = parse_csv(current_solution_path)

# Calculate current total score
current_score = get_total_score(dict_of_side_length)

In [ ]:
for group_id_main in range(200, 2, -1):
    group_id_main = f'{int(group_id_main):03n}'
    print(f'\nCurrent box: {group_id_main}')

    candidate_tree_list = [tree.clone() for tree in dict_of_tree_list[group_id_main]]

    while len(candidate_tree_list) > 1:
        group_id_prev = f'{len(candidate_tree_list) - 1:03n}'
        best_side_length = dict_of_side_length[group_id_prev]
        best_side_length_temp = 100
        best_tree_idx_to_delete = None

        # Try to delete each tree one by one and select the best option
        tree_idx_list = get_bbox_touching_tree_indices(candidate_tree_list)      
        for tree_idx_to_delete in tree_idx_list:
            
            candidate_tree_list_short = [tree.clone() for tree in candidate_tree_list]
            del candidate_tree_list_short[tree_idx_to_delete]
    
            candidate_side_length = get_tree_list_side_lenght(candidate_tree_list_short)
                
            if candidate_side_length < best_side_length_temp:
                best_side_length_temp = candidate_side_length
                best_tree_idx_to_delete = tree_idx_to_delete

        # Save the best
        if best_tree_idx_to_delete is not None:
            # print(F'   Deleting: {best_tree_idx_to_delete}')
            del candidate_tree_list[best_tree_idx_to_delete]
            print(len(candidate_tree_list), end=' ')

            if candidate_side_length < best_side_length:
                print(f'\nimprovement {best_side_length:0.8f} -> {candidate_side_length:0.8f}')
            
                dict_of_tree_list[group_id_prev] = [tree.clone() for tree in candidate_tree_list]
                dict_of_side_length[group_id_prev] = get_tree_list_side_lenght(dict_of_tree_list[group_id_prev])

        if int(group_id_main) - int(group_id_prev) > 5:
            break

    # break

    
# Recalculate current total score
new_score = get_total_score(dict_of_side_length)
print(f'\n{current_score=:0.8f} {new_score=:0.8f} ({current_score - new_score:0.8f})')

In [ ]:
# Save results
tree_data = []
for group_name, tree_list in dict_of_tree_list.items():
    for item_id, tree in enumerate(tree_list):
        tree_data.append({
            'id': f'{group_name}_{item_id}',
            'x': f's{tree.center_x}',
            'y': f's{tree.center_y}',
            'deg': f's{tree.angle}'
        })
tree_data = pd.DataFrame(tree_data)
tree_data.to_csv('submission.csv', index=False)   